In [ ]:
md = """
## Introduction

Datapane is a Python library which makes it easy to create and share interactive reports. You can analyze and visualize
your data using your favourite Python libraries, and quickly build and share a report on the web or with your team.

In this tutorial, we're going to show you how to use `pandas`, `altair`, and `datapane` to build a dashboard
of COVID vaccinations. Follow along with the code snippets here. You can find the full code on
the "Source code" tab of this report, and the final dashboard on the "Dashboard" tab.

Before getting started, install the `datapane` library from pip and [create a free Datapane account](https://datapane.com/accounts/signup/) so you can publish your report.
```
$ pip3 install -U datapane
```

-------

## Analyzing our data

First, we will fetch the vaccination data from
OurWorldInData and will use Datapane to create a report with visualizations and data.

Let's import the libraries and download our dataset:

```python
import pandas as pd
import altair as alt

df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv')
```

Now that we have our dataset, we can take a look to see which we can plot.

{{}}

If we want to compare a few countries in our Datapane report, what do we have to choose from?

Let's find the unique locations using `df['location'].unique()`.

{{}}

We will pick a few we're interested in and filter the dataframe.

```python
countries = ['United Kingdom', 'Germany', 'France']

subset = df[df.location.isin(countries)]
```
{{}}

Our data is now a more manageable size for visualizing in our report. 
For our visualization, let's create a plot for `total_vaccinations_per_hundred`, `daily_vaccinations_per_million`, and `people_vaccinated`.

## Creating our visualizations

We're going to use the Python library Altair to create our visualizations. A great thing about Altair is that we can create a common base which we reuse across the multiple plots in our report.

In the following Python code, we're setting a bit of stroke and adjusting the opacity to make our report a bit more visually appealing. 
We're also choosing to make it interactive, so our users can zoom in and out. Datapane will make the plots responsive for us automatically when we create our report.

```python
base = alt.Chart(df).encode(
    x='date:T',
    color='location'
).mark_area(
    opacity=0.5, 
    stroke='black'
).interactive()
```

Once we have our base, we only need to choose the y axis for each (remember, we can put the x axis in our base, because every plot uses date.)

```python
c1 = base.encode(y='total_vaccinations_per_hundred')
c2 = base.encode(y='daily_vaccinations_per_million')
c3 = base.encode(y='people_vaccinated')
```

Let's preview what these plots look like:

{{}}

## Creating our report

Now we have some plots and a DataFrame, we can build our report using datapane. 
Datapane provides components which wrap around the different plots and datasets inside our notebook, 
such as `DataTable` and `Plot`. We're also using the `Group` component, which allows us to place components in a column or grid layout. 
`DataTable` and `Plot` both have an optional caption parameter where we can provide some further info.

```python
report = dp.Report(
  dp.DataTable(df, caption=f'Dataset for {countries}'),
  dp.Group(dp.Plot(c1, caption='Vaccination %'), dp.Plot(c2, caption='Daily vaccinations per million'), columns=2),
  dp.Plot(c3, caption='Total vaccinations')
)
```

-------

## Sharing our report

When we're happy with our report, we can publish it to Datapane.com to host it online.

If you haven't already, make sure you've logged into your Datapane API with your token before publishing (you can find your token in your [settings](https://datapane.com/settings) )

```
$ datapane login --token=...
```

```python
report.publish(name='My Covid Report', open=True)
```

That's it! As you can see, building a report on Datapane only takes a few lines of code. In fact, this tutorial itself was built using Datapane too - interested in seeing how? Download it here:
{{}}

"""

# Code for assets
import pandas as pd
import altair as alt
import datapane as dp

df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv')
countries = ['United Kingdom', 'Germany', 'France']

subset = df[df.location.isin(countries)]

base = alt.Chart(subset).encode(
    x='date:T',
    color='location'
).mark_line(
    size=5,
    opacity=0.75, 
).interactive()

c1 = base.encode(y='total_vaccinations_per_hundred')
c2 = base.encode(y='daily_vaccinations_per_million')
c3 = base.encode(y='people_vaccinated')

finished_code = """import pandas as pd
import altair as alt
import datapane as dp

df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv')
countries = ['United Kingdom', 'Germany', 'France']

subset = df[df.location.isin(countries)]

base = alt.Chart(subset).encode(
    x='date:T',
    color='location'
).mark_line(
    size=5,
    opacity=0.75, 
).interactive()

plots = [
  base.encode(y='total_vaccinations_per_hundred'), 
  base.encode(y='daily_vaccinations_per_million'),
  base.encode(y='people_vaccinated')
]

report = dp.Report(
    dp.Text(f"## Covid vaccinations in {', '.join(countries)}"),
    dp.Group(*plots[:2], columns=2),
    plots[2],
    dp.DataTable(subset, caption=f'Dataset for {countries}'),
)

report.publish(name='My Covid Report', open=True)"""

plots = [
    dp.Plot(c1, label='% Population vaccinated'), 
    dp.Plot(c2, label='Daily vaccinations (per million)'), 
    dp.Plot(c3, label='People vaccinations')
]

objects = [
    dp.DataTable(df),
    dp.DataTable(pd.DataFrame({"columns": df['location'].unique()})),
    dp.DataTable(subset),
    dp.Select(*plots),
    dp.File(file="./Notebook Tutorial 1.ipynb", label='Jupyter Notebook')
]

dp.Report(
    dp.Page(label='Tutorial', blocks=[dp.Text(md).format(*objects)]),
    dp.Page(label='Source code', blocks=[dp.Code(finished_code, label='Python code')]),
    dp.Page(label='Dashboard', blocks=[    
            dp.Group(blocks=[
                dp.Text(f"## Covid vaccinations in {', '.join(countries)}"),
                dp.Group(*plots[:2], columns=2),
                plots[2],
                dp.DataTable(subset, caption=f'Dataset for {countries}'),
            ]
        ),
    ])
).publish(name='Tutorial #1')

In [ ]:
df.describe()